In [ ]:
"""Script to calculate head movement in mm between runs"""


import numpy as np
import pandas as pd
import os
import glob
import mne

subjects = [f"S{i:02}" for i in range(1, 21)]  # S01 to S20
#Asign path to bids directory
bids_top_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
raw_ICA_path = bids_top_dir + 'derivatives/preprocessed/ICA-Removed'
output_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/!Important Data/Head_Displacement' #Output path
os.makedirs(output_dir, exist_ok=True)

for S in subjects:
    print(f"\nProcessing Subject {S}...")

    # Load static runs
    static_runs = []
    for run_i in range(1, 5):
        pattern = f'{raw_ICA_path}/sub-{S}_ses-1_task-OrientationImagery_run-{run_i:02}_step1b-raw.fif'
        matches = glob.glob(pattern)
        if matches:
            static_runs.append(matches[0])
        else:
            print(f"WARNING: Missing static run {run_i} for {S}")
    if len(static_runs) < 4:
        print(f"  WARNING: Only found {len(static_runs)} static runs for {S}, skipping subject.")
        continue

    # Load dynamic runs
    dynamic_runs = []
    for run_i in range(1, 5):
        pattern = f'{raw_ICA_path}/sub-{S}_ses-1_task-OrientationImageryDynamic_run-{run_i:02}_step1b-raw.fif'
        matches = glob.glob(pattern)
        if matches:
            dynamic_runs.append(matches[0])
        else:
            print(f"  WARNING: Missing dynamic run {run_i} for {S}")
    if len(dynamic_runs) < 4:
        print(f"  WARNING: Only found {len(dynamic_runs)} dynamic runs for {S}, skipping subject.")
        continue

    # Load in epochs and append to a list
    all_epochs = []
    event_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

    for fpath in static_runs:
        raw = mne.io.read_raw_fif(fpath, verbose=False)
        events = mne.events_from_annotations(raw)[0]
        pre_stim_time = -.2
        post_stim_time = .6
        epochs = mne.Epochs(raw, events, event_id=event_id, tmin=pre_stim_time, tmax=post_stim_time,
                            preload=True, baseline=(-0.2, 0), picks='mag', verbose=False)
        all_epochs.append(epochs)

    all_epochsDynamic = []
    for fpath in dynamic_runs:
        raw = mne.io.read_raw_fif(fpath, verbose=False)
        events = mne.events_from_annotations(raw)[0]
        pre_stim_time = -.2
        post_stim_time = 3
        epochs = mne.Epochs(raw, events, event_id=event_id, tmin=pre_stim_time, tmax=post_stim_time,
                            preload=True, baseline=(None, 0), picks='mag', verbose=False)
        all_epochsDynamic.append(epochs)

    # Put runs in correct order by interleaving them
    combined_epochs = []
    for i in range(min(len(all_epochs), len(all_epochsDynamic))):
        combined_epochs.append(all_epochs[i])
        combined_epochs.append(all_epochsDynamic[i])

    # Get head position for each run
    head_positions_mm = []
    for run_idx, ep in enumerate(combined_epochs):
        trans = ep.info['dev_head_t']['trans'][:3, 3] * 1000  # meters to mm
        head_positions_mm.append(trans)

    # Determine displacement from previous runs
    ref_pos = head_positions_mm[0]

    displacements_from_run1 = [np.linalg.norm(pos - ref_pos) for pos in head_positions_mm]
    displacements_from_prev = [np.nan]  # no previous run for Run 1
    for i in range(1, len(head_positions_mm)):
        disp = np.linalg.norm(head_positions_mm[i] - head_positions_mm[i-1])
        displacements_from_prev.append(disp)

    # Save out results to dataframe
    runs = [f'Run_{i+1}' for i in range(len(head_positions_mm))]
    headpos_array = np.array(head_positions_mm)

    df = pd.DataFrame({
        'Run': runs,
        'HeadPos_X_mm': headpos_array[:, 0],
        'HeadPos_Y_mm': headpos_array[:, 1],
        'HeadPos_Z_mm': headpos_array[:, 2],
        'Displacement_from_Run1_mm': displacements_from_run1,
        'Displacement_from_Previous_Run_mm': displacements_from_prev
    })

    #out_csv = os.path.join(output_dir, f'sub-{S}_headpos_displacement.csv')
    #df.to_csv(out_csv, index=False)
    #print(f"Saved displacement data for {S} to {out_csv}")



Processing Subject S01...
Used Annotations descriptions: ['S01/Still/0001', 'S01/Still/0022', 'S01/Still/0045', 'S01/Still/0067', 'S01/Still/0090', 'S01/Still/0112', 'S01/Still/0135', 'S01/Still/0157', 'S01/Still/0180', 'S01/Still/0202', 'S01/Still/0225', 'S01/Still/0247', 'S01/Still/0270', 'S01/Still/0292', 'S01/Still/0315', 'S01/Still/0337', 'S01/Still/catch']
Used Annotations descriptions: ['S01/Still/0001', 'S01/Still/0022', 'S01/Still/0045', 'S01/Still/0067', 'S01/Still/0090', 'S01/Still/0112', 'S01/Still/0135', 'S01/Still/0157', 'S01/Still/0180', 'S01/Still/0202', 'S01/Still/0225', 'S01/Still/0247', 'S01/Still/0270', 'S01/Still/0292', 'S01/Still/0315', 'S01/Still/0337', 'S01/Still/catch']
Used Annotations descriptions: ['S01/Still/0001', 'S01/Still/0022', 'S01/Still/0045', 'S01/Still/0067', 'S01/Still/0090', 'S01/Still/0112', 'S01/Still/0135', 'S01/Still/0157', 'S01/Still/0180', 'S01/Still/0202', 'S01/Still/0225', 'S01/Still/0247', 'S01/Still/0270', 'S01/Still/0292', 'S01/Still/

KeyboardInterrupt: 